# Artifact of Accessibility Issues in Ad-Driven Web Applications
Artifact of the paper "Accessibility Issues in Ad-Driven Web Applications" from ICSE 2025.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [1]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [2]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 46 compute_cascadelake_r nodes ready to use


### Reserve node

In [3]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start... This can take up to 60 seconds


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [4]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,c9bdf4bd-f0b4-4466-8b66-3b3dec476efb
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.0.55 (v4) Type: fixed MAC: bc:97:e1:78:f1:a0
Network Name,sharednet1
Created At,2025-03-26T09:24:19Z
Keypair,trovi-0e73ceb
Reservation Id,09bb381b-4b2d-4dea-b274-98e3248b2108
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [5]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.67 port 22.


Connection successful


## Run Experiment

Now, we can finally run the experiment. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [36]:
my_server.execute("rm -rf Web-Ads-Accessibility && git clone https://github.com/radhofan/Web-Ads-Accessibility.git")

Cloning into 'Web-Ads-Accessibility'...


<Result cmd='rm -rf Web-Ads-Accessibility && git clone https://github.com/radhofan/Web-Ads-Accessibility.git' exited=0>

In [22]:
my_server.execute("""
# 1. Install missing dependencies for Chrome
echo "Installing missing dependencies..."

sudo add-apt-repository universe
sudo add-apt-repository multiverse
sudo apt-get update

sudo apt update >/dev/null 2>&1
sudo apt-get install -y libatk1.0-0 libatk-bridge2.0-0 libgdk-pixbuf2.0-0 libgtk-3-0 libx11-6 libxcomposite1 libxdamage1 libxext6 libxfixes3 libxi6 libxrandr2 libxrender1 libxtst6 libnss3 libnspr4 libgconf-2-4 >/dev/null 2>&1
# sudo apt install -y wget unzip libatk-1.0-0 libatk-bridge2.0-0 libgtk-3-0 libgbm1 libnss3 libxss1 libasound2 >/dev/null 2>&1

# Verify installation of critical libraries
if ! dpkg -l | grep -q libatk1.0-0; then
    echo "Error: libatk-1.0-0 is not installed. Reinstalling..."
    sudo apt install -y libatk1.0-0
fi

# Update dynamic linker cache
sudo ldconfig

# 2. Remove existing Chrome and ChromeDriver installations (forceful)
echo "Removing existing Chrome and ChromeDriver installations..."
sudo apt purge -y google-chrome* chromium* >/dev/null 2>&1
sudo rm -rf /usr/bin/google-chrome /usr/local/bin/chromedriver /opt/google/chrome /usr/local/chrome /usr/local/chromedriver >/dev/null 2>&1

# 3. Create necessary directories
echo "Creating necessary directories..."
sudo mkdir -p /usr/local/chrome /usr/local/chromedriver
sudo chmod -R 755 /usr/local/chrome /usr/local/chromedriver

# 4. Install Chrome (with download verification)
CHROME_VERSION="134.0.6998.165"
CHROME_URL="https://storage.googleapis.com/chrome-for-testing-public/${CHROME_VERSION}/linux64/chrome-linux64.zip"

echo "Downloading Chrome version ${CHROME_VERSION}..."
if ! wget -q "$CHROME_URL"; then
    echo "Failed to download Chrome from $CHROME_URL" >&2
    exit 1
fi

echo "Extracting Chrome..."
sudo unzip -o chrome-linux64.zip -d /usr/local/chrome/
sudo chmod +x /usr/local/chrome/chrome-linux64/chrome
sudo ln -sf /usr/local/chrome/chrome-linux64/chrome /usr/bin/google-chrome

# 5. Install ChromeDriver (with download verification)
CHROMEDRIVER_URL="https://storage.googleapis.com/chrome-for-testing-public/${CHROME_VERSION}/linux64/chromedriver-linux64.zip"

echo "Downloading ChromeDriver version ${CHROME_VERSION}..."
if ! wget -q "$CHROMEDRIVER_URL"; then
    echo "Failed to download ChromeDriver from $CHROMEDRIVER_URL" >&2
    exit 1
fi

echo "Extracting ChromeDriver..."
sudo unzip -o chromedriver-linux64.zip -d /usr/local/chromedriver/

# Verify that the chromedriver binary exists
if [ ! -f "/usr/local/chromedriver/chromedriver-linux64/chromedriver" ]; then
    echo "ChromeDriver binary not found after extraction!" >&2
    exit 1
fi

# Move the binary to the correct location
sudo mv /usr/local/chromedriver/chromedriver-linux64/chromedriver /usr/local/chromedriver/
sudo chmod +x /usr/local/chromedriver/chromedriver
sudo ln -sf /usr/local/chromedriver/chromedriver /usr/local/bin/chromedriver

# 6. Verify installations
echo "Verifying installations..."
echo "Chrome path: $(which google-chrome)"
echo "Chrome version: $(google-chrome --version)"
echo "ChromeDriver path: $(which chromedriver)"
chromedriver --version

# 7. Cleanup temporary files
echo "Cleaning up temporary files..."
rm -f chrome-linux64.zip chromedriver-linux64.zip

echo "Installation completed successfully!"
""")

Installing missing dependencies...
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 525 kB in 1s (981 kB/s)
Reading package lists...
Adding component(s) 'universe' to all repositories.
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 270 kB in 1s (264 kB/s)
Reading package lists...
Adding component(s) 'multiverse' to all repositories.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease


<Result cmd='\n# 1. Install missing dependencies for Chrome\necho "Installing missing dependencies..."\n\nsudo add-apt-repository universe\nsudo add-apt-repository multiverse\nsudo apt-get update\n\nsudo apt update >/dev/null 2>&1\nsudo apt-get install -y libatk1.0-0 libatk-bridge2.0-0 libgdk-pixbuf2.0-0 libgtk-3-0 libx11-6 libxcomposite1 libxdamage1 libxext6 libxfixes3 libxi6 libxrandr2 libxrender1 libxtst6 libnss3 libnspr4 libgconf-2-4 >/dev/null 2>&1\n# sudo apt install -y wget unzip libatk-1.0-0 libatk-bridge2.0-0 libgtk-3-0 libgbm1 libnss3 libxss1 libasound2 >/dev/null 2>&1\n\n# Verify installation of critical libraries\nif ! dpkg -l | grep -q libatk1.0-0; then\n    echo "Error: libatk-1.0-0 is not installed. Reinstalling..."\n    sudo apt install -y libatk1.0-0\nfi\n\n# Update dynamic linker cache\nsudo ldconfig\n\n# 2. Remove existing Chrome and ChromeDriver installations (forceful)\necho "Removing existing Chrome and ChromeDriver installations..."\nsudo apt purge -y google-chro

In [8]:
my_server.execute("""
curl -L -o IBM-Equal-Access-Accessibility-Checker.zip \
     https://github.com/IBMa/equal-access/archive/refs/tags/4.0.2.zip
""")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0-- --:--:--     0
100  100M    0  100M    0     0  8730k      0 --:--:--  0:00:11 --:--:-- 8019M.8M      0 --:--:--  0:00:02 --:--:-- 12.9M1M      0 --:--:--  0:00:03 --:--:-- 11.3M9806k      0 --:--:--  0:00:04 --:--:-- 10.3M.6M    0     0  9470k      0 --:--:--  0:00:05 --:--:--  9.8M   0 62.4M    0     0  9229k      0 --:--:--  0:00:06 --:--:-- 7932k    0 70.1M    0     0  9080k      0 --:--:--  0:00:07 --:--:-- 7898k   0 77.6M    0     0  8895k      0 --:--:--  0:00:08 --:--:-- 7744k  0 84.9M    0     0  8690k      0 --:--:--  0:00:10 --:--:-- 7583k0:10 --:--:-- 7487k8k


<Result cmd='\ncurl -L -o IBM-Equal-Access-Accessibility-Checker.zip      https://github.com/IBMa/equal-access/archive/refs/tags/4.0.2.zip\n' exited=0>

In [35]:
my_server.execute("""
curl -L -o uBlock-Origin.zip \
     https://github.com/gorhill/uBlock/archive/refs/tags/1.50.0.zip
""")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0   0    0     0      0      0 --:--:-- --:--:-- --:--:--     0--     0
100 4139k    0 4139k    0     0  4795k      0 --:--:-- --:--:-- --:--:-- 4795k


<Result cmd='\ncurl -L -o uBlock-Origin.zip      https://github.com/gorhill/uBlock/archive/refs/tags/1.50.0.zip\n' exited=0>

In [37]:
my_server.execute("chmod +x Web-Ads-Accessibility/reproduce.sh")
my_server.execute("bash Web-Ads-Accessibility/reproduce.sh")

ERROR: File or directory already exists: '/home/cc/miniconda'
If you want to update an existing installation, use the -u option.


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.

Running `shell init`, which:
 - modifies RC file: "/home/cc/.bashrc"
 - generates config for root prefix: "/home/cc/miniconda"
 - sets mamba executable to: "/home/cc/miniconda/bin/mamba"
The following has been added in your "/home/cc/.bashrc" file

# >>> mamba initialize >>>
# !! Contents within this block are managed by 'mamba shell init' !!
export MAMBA_EXE='/home/cc/miniconda/bin/mamba';
export MAMBA_ROOT_PREFIX='/home/cc/miniconda';
__mamba_setup="$("$MAMBA_EXE" shell hook --shell bash --root-prefix "$MAMBA_ROOT_PREFIX" 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__mamba_setup"
else
    alias mamba="$MAMBA_EXE"  # Fallback on help from mamba activate
fi
unset __mamba_setup
# <<< mamba initialize <<<

Empty environment created at prefix: /home/cc/miniconda/envs/icse


Traceback (most recent call last):
  File "/home/cc/Web-Ads-Accessibility/src/sele.py", line 204, in <module>
    crawl_ibm_extension(website, adblock=False)
  File "/home/cc/Web-Ads-Accessibility/src/sele.py", line 94, in crawl_ibm_extension
    driver = webdriver.Chrome(service=service, options=chrome_options)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/selenium/webdriver/chrome/webdriver.py", line 45, in __init__
    super().__init__(
  File "/home/cc/miniconda/lib/python3.12/site-packages/selenium/webdriver/chromium/webdriver.py", line 66, in __init__
    super().__init__(command_executor=executor, options=options)
  File "/home/cc/miniconda/lib/python3.12/site-packages/selenium/webdriver/remote/webdriver.py", line 250, in __init__
    self.start_session(capabilities)
  File "/home/cc/miniconda/lib/python3.12/site-packages/selenium/webdriver/remote/webdriver.py", line 342, in start_session
    respon

Crawling alaskanewssource.com with ads
alaskanewssource.com crashed
Error details:


Traceback (most recent call last):
  File "/home/cc/Web-Ads-Accessibility/src/sele_adblock.py", line 126, in <module>
    crawl_ibm_extension(website, adblock=True)
  File "/home/cc/Web-Ads-Accessibility/src/sele_adblock.py", line 52, in crawl_ibm_extension
    driver = webdriver.Chrome(service=service, options=chrome_options)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/selenium/webdriver/chrome/webdriver.py", line 45, in __init__
    super().__init__(
  File "/home/cc/miniconda/lib/python3.12/site-packages/selenium/webdriver/chromium/webdriver.py", line 66, in __init__
    super().__init__(command_executor=executor, options=options)
  File "/home/cc/miniconda/lib/python3.12/site-packages/selenium/webdriver/remote/webdriver.py", line 250, in __init__
    self.start_session(capabilities)
  File "/home/cc/miniconda/lib/python3.12/site-packages/selenium/webdriver/remote/webdriver.py", line 342, in start_ses

...

----------------------Figure 8b--------------------


    normal_scores = pd.read_csv('normal_8b.csv', header=None).squeeze()
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1880, in _make_engine
    self.handles = get_handle(
                   ^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pand

----------------------Figure 8c--------------------


Traceback (most recent call last):
  File "/home/cc/Web-Ads-Accessibility/src/reproducible/fig8c.py", line 10, in <module>
    normal_scores = pd.read_csv('normal_8c.csv', header=None).squeeze()
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cc/miniconda/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1880, in _make_engine

UnexpectedExit: Encountered a bad command exit code!

Command: 'bash Web-Ads-Accessibility/reproduce.sh'

Exit code: 1

Stdout: already printed

Stderr: already printed

